In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pds

In [5]:
def webScrape(links):
  unique_urls = set()
  for link in links:
    response = requests.get(link)
    html_content = response.content
    soup = BeautifulSoup(html_content, 'html.parser')
    buttons = soup.find_all('a')
    url = {button['href'] for button in buttons if 'href' in button.attrs}
    unique_urls.update(url)

    for link in url:
        print(link)
    print(f"Total Links: {len(unique_urls)}")
  return unique_urls

In [6]:
url = ['https://results.eci.gov.in/']
urls = webScrape(url)

index.htm
https://results.eci.gov.in/PcResultGenJune2024/index.htm
https://results.eci.gov.in/AcResultByeJune2024/
https://results.eci.gov.in/AcResultGenJune2024/index.htm
https://apps.apple.com/in/app/voter-helpline/id1456535004
https://play.google.com/store/apps/details?id=com.eci.citizen
https://results.eci.gov.in/AcResultGen2ndJune2024/index.htm
Total Links: 7


In [7]:
filtered_urls = [url for url in urls if url.endswith('/index.htm')]
print(filtered_urls)

['https://results.eci.gov.in/PcResultGenJune2024/index.htm', 'https://results.eci.gov.in/AcResultGen2ndJune2024/index.htm', 'https://results.eci.gov.in/AcResultGenJune2024/index.htm']


In [8]:
def fetch_data(url):
    response = requests.get(url)
    if response.status_code == 200:
        page_content = response.content
        soup = BeautifulSoup(page_content, 'html.parser')
        return soup
    else:
        print(f"Failed to retrieve {url}")
        return None

def categorize_tables(soup, url):
    class_table = []
    non_class_table = []
    
    tables = soup.find_all('table')
    for table in tables:
        if 'table' in table.get('class', []):
            class_table.append(url) 
        else:
            non_class_table.append(table)
    
    return class_table, non_class_table

def extract_data(tables, category):
    print(f"\nExtracting data from {category} tables...")
    for table in tables:
        print(table) 

# Traverse each URL
for url in filtered_urls:
    print(f"Fetching data from {url}")
    soup = fetch_data(url)
    if soup:
        class_tables, non_class_tables = categorize_tables(soup, url)
        
        # Print the URLs of class tables
        print(f"Class tables found in {url}:")
        print()
        
        for table_url in class_tables:
            print(table_url)


Fetching data from https://results.eci.gov.in/PcResultGenJune2024/index.htm
Class tables found in https://results.eci.gov.in/PcResultGenJune2024/index.htm:

https://results.eci.gov.in/PcResultGenJune2024/index.htm
Fetching data from https://results.eci.gov.in/AcResultGen2ndJune2024/index.htm
Class tables found in https://results.eci.gov.in/AcResultGen2ndJune2024/index.htm:

Fetching data from https://results.eci.gov.in/AcResultGenJune2024/index.htm
Class tables found in https://results.eci.gov.in/AcResultGenJune2024/index.htm:



In [9]:
print(class_tables)

[]


In [16]:
import csv
import os

def fetch_data(url):
    response = requests.get(url)
    if response.status_code == 200:
        page_content = response.content
        soup = BeautifulSoup(page_content, 'html.parser')
        return soup
    else:
        print(f"Failed to retrieve {url}")
        return None

def extract_data_to_csv(soup, filename):
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        tables = soup.find_all('table')
        for table in tables:
            if table.find('th'): 
                headers = [th.text.strip() for th in table.find_all('th')]
                writer.writerow(headers)

                rows = table.find_all('tr')[1:]  
                for row in rows:
                    columns = row.find_all('td')
                    row_data = [column.text.strip() for column in columns]
                    writer.writerow(row_data)

output_folder = 'csv_file'
os.makedirs(output_folder, exist_ok=True)

for url in class_tables:
    print(f"Fetching data from {url}")
    soup = fetch_data(url)
    if soup:
        filename = os.path.join(output_folder, f"{url.split('/')[-2]}.csv")  
        extract_data_to_csv(soup, filename)
        print(f"Data written to {filename}")


Fetching data from https://results.eci.gov.in/PcResultGenJune2024/index.htm
Data written to csv_file\PcResultGenJune2024.csv


In [4]:
url = 'https://results.eci.gov.in/PcResultGenJune2024/index.htm'

response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
dropdown = soup.find('select', id='ctl00_ContentPlaceHolder1_Result1_ddlState')
    

In [5]:
print(dropdown)

<select id="ctl00_ContentPlaceHolder1_Result1_ddlState" name="state" onchange="return GetResult(this)"> <option value=""> Select State Wise </option><option value="U01">Andaman &amp; Nicobar Islands</option><option value="S01">Andhra Pradesh</option><option value="S02">Arunachal Pradesh</option><option value="S03">Assam</option><option value="S04">Bihar</option><option value="U02">Chandigarh</option><option value="S26">Chhattisgarh</option><option value="U03">Dadra &amp; Nagar Haveli and Daman &amp; Diu</option><option value="S05">Goa</option><option value="S06">Gujarat</option><option value="S07">Haryana</option><option value="S08">Himachal Pradesh</option><option value="U08">Jammu and Kashmir</option><option value="S27">Jharkhand</option><option value="S10">Karnataka</option><option value="S11">Kerala</option><option value="U09">Ladakh</option><option value="U06">Lakshadweep</option><option value="S12">Madhya Pradesh</option><option value="S13">Maharashtra</option><option value="S14"

In [6]:
state_links = {}
if dropdown:
    options = dropdown.find_all('option')
    for option in options:
        option_value = option.get('value')
        if option_value:  # Check if option_value is not None or empty
            state_name = option.text.strip()
            state_links[state_name] = 'https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-' + option_value + '.htm'
else:
    print("Dropdown not found.")

# Print the dictionary to see the result
for state, link in state_links.items():
    print(f'{state}: {link}')

Andaman & Nicobar Islands: https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U01.htm
Andhra Pradesh: https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S01.htm
Arunachal Pradesh: https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S02.htm
Assam: https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S03.htm
Bihar: https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S04.htm
Chandigarh: https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U02.htm
Chhattisgarh: https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S26.htm
Dadra & Nagar Haveli and Daman & Diu: https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U03.htm
Goa: https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S05.htm
Gujarat: https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S06.htm
Haryana: https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S07.htm
Himachal Pradesh: https://results.eci.gov.in/PcResultGen

In [14]:
import requests
from bs4 import BeautifulSoup
import csv
import os

def fetch_data(url):
    response = requests.get(url)
    if response.status_code == 200:
        page_content = response.content
        soup = BeautifulSoup(page_content, 'html.parser')
        return soup
    else:
        print(f"Failed to retrieve {url}")
        return None

def extract_data_to_csv(soup, filename):
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        tables = soup.find_all('table')
        for table in tables:
            if table.find('th'): 
                headers = [th.text.strip() for th in table.find_all('th')]
                writer.writerow(headers)

                rows = table.find_all('tr')[1:]  
                for row in rows:
                    columns = row.find_all('td')
                    row_data = [column.text.strip() for column in columns]
                    writer.writerow(row_data)

output_folder = 'csv_statefiles'
os.makedirs(output_folder, exist_ok=True)

for state_name, url in state_links.items():
    print(f"Fetching data from {url}")
    soup = fetch_data(url)
    if soup:
        # Replace spaces with underscores and remove any special characters for the filename
        safe_state_name = state_name.replace(" ", "_").replace("&", "and")
        filename = os.path.join(output_folder, f"{safe_state_name}.csv")
        extract_data_to_csv(soup, filename)
        print(f"Data written to {filename}")

Fetching data from https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U01.htm
Data written to csv_statefiles\Andaman_and_Nicobar_Islands.csv
Fetching data from https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S01.htm
Data written to csv_statefiles\Andhra_Pradesh.csv
Fetching data from https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S02.htm
Data written to csv_statefiles\Arunachal_Pradesh.csv
Fetching data from https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S03.htm
Data written to csv_statefiles\Assam.csv
Fetching data from https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S04.htm
Data written to csv_statefiles\Bihar.csv
Fetching data from https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U02.htm
Data written to csv_statefiles\Chandigarh.csv
Fetching data from https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S26.htm
Data written to csv_statefiles\Chhattisgarh.csv
Fetching data from https://r

In [11]:
import os

# Main output folder
main_output_folder = 'state_folders'
os.makedirs(main_output_folder, exist_ok=True)

# Create subfolders for each state/UT
for state_name in state_links.keys():
    # Replace spaces with underscores and remove any special characters for the folder name
    safe_state_name = state_name.replace(" ", "_").replace("&", "and")
    state_folder = os.path.join(main_output_folder, safe_state_name)
    
    # Create a subfolder for the state
    os.makedirs(state_folder, exist_ok=True)
    print(f"Folder created: {state_folder}")


Folder created: state_folders\Andaman_and_Nicobar_Islands
Folder created: state_folders\Andhra_Pradesh
Folder created: state_folders\Arunachal_Pradesh
Folder created: state_folders\Assam
Folder created: state_folders\Bihar
Folder created: state_folders\Chandigarh
Folder created: state_folders\Chhattisgarh
Folder created: state_folders\Dadra_and_Nagar_Haveli_and_Daman_and_Diu
Folder created: state_folders\Goa
Folder created: state_folders\Gujarat
Folder created: state_folders\Haryana
Folder created: state_folders\Himachal_Pradesh
Folder created: state_folders\Jammu_and_Kashmir
Folder created: state_folders\Jharkhand
Folder created: state_folders\Karnataka
Folder created: state_folders\Kerala
Folder created: state_folders\Ladakh
Folder created: state_folders\Lakshadweep
Folder created: state_folders\Madhya_Pradesh
Folder created: state_folders\Maharashtra
Folder created: state_folders\Manipur
Folder created: state_folders\Meghalaya
Folder created: state_folders\Mizoram
Folder created: st

In [21]:
def fetch_data(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.content
    else:
        print(f"Failed to retrieve {url}")
        return None

state_party_links = {}

for state_name, url in state_links.items():
    print(f"Fetching party links for {state_name} from {url}")
    
    page_content = fetch_data(url)
    if page_content:
        soup = BeautifulSoup(page_content, 'html.parser')
        
        party_rows = soup.select('tbody tr.tr')
        
        party_list = []

        for row in party_rows:
            party_name = row.find('td', style='text-align:left').text.strip()
            party_link = row.find('a')['href']
            party_list.append((party_name, party_link))

        state_party_links[state_name] = party_list
    
    print(f"Party links fetched for {state_name}")
    print("-" * 50)

print("State Party Links:")
for state, parties in state_party_links.items():
    print(f"State: {state}")
    for party_name, party_link in parties:
        print(f"Party Name: {party_name}, Link: {party_link}")
    print("-" * 50)

Fetching party links for Andaman & Nicobar Islands from https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U01.htm
Party links fetched for Andaman & Nicobar Islands
--------------------------------------------------
Fetching party links for Andhra Pradesh from https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S01.htm
Party links fetched for Andhra Pradesh
--------------------------------------------------
Fetching party links for Arunachal Pradesh from https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S02.htm
Party links fetched for Arunachal Pradesh
--------------------------------------------------
Fetching party links for Assam from https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S03.htm
Party links fetched for Assam
--------------------------------------------------
Fetching party links for Bihar from https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S04.htm
Party links fetched for Bihar
-----------------------------

In [17]:
print(state_party_links)

{'Andaman & Nicobar Islands': [('Bharatiya Janata Party - BJP', 'partywisewinresult-369U01.htm')], 'Andhra Pradesh': [('Telugu Desam - TDP', 'partywisewinresult-1745S01.htm'), ('Yuvajana Sramika Rythu Congress Party - YSRCP', 'partywisewinresult-1888S01.htm'), ('Bharatiya Janata Party - BJP', 'partywisewinresult-369S01.htm'), ('Janasena Party - JnP', 'partywisewinresult-860S01.htm')], 'Arunachal Pradesh': [('Bharatiya Janata Party - BJP', 'partywisewinresult-369S02.htm')], 'Assam': [('Bharatiya Janata Party - BJP', 'partywisewinresult-369S03.htm'), ('Indian National Congress - INC', 'partywisewinresult-742S03.htm'), ('United People’s Party, Liberal - UPPL', 'partywisewinresult-1998S03.htm'), ('Asom Gana Parishad - AGP', 'partywisewinresult-83S03.htm')], 'Bihar': [('Janata Dal  (United) - JD(U)', 'partywisewinresult-805S04.htm'), ('Bharatiya Janata Party - BJP', 'partywisewinresult-369S04.htm'), ('Lok Janshakti Party(Ram Vilas) - LJPRV', 'partywisewinresult-3165S04.htm'), ('Rashtriya Ja

In [18]:
base_url = "https://results.eci.gov.in/PcResultGenJune2024/"

for state, parties in state_party_links.items():
    updated_parties = [(party, base_url + link) for party, link in parties]
    state_party_links[state] = updated_parties

print(state_party_links)

{'Andaman & Nicobar Islands': [('Bharatiya Janata Party - BJP', 'https://results.eci.gov.in/PcResultGenJune2024/partywisewinresult-369U01.htm')], 'Andhra Pradesh': [('Telugu Desam - TDP', 'https://results.eci.gov.in/PcResultGenJune2024/partywisewinresult-1745S01.htm'), ('Yuvajana Sramika Rythu Congress Party - YSRCP', 'https://results.eci.gov.in/PcResultGenJune2024/partywisewinresult-1888S01.htm'), ('Bharatiya Janata Party - BJP', 'https://results.eci.gov.in/PcResultGenJune2024/partywisewinresult-369S01.htm'), ('Janasena Party - JnP', 'https://results.eci.gov.in/PcResultGenJune2024/partywisewinresult-860S01.htm')], 'Arunachal Pradesh': [('Bharatiya Janata Party - BJP', 'https://results.eci.gov.in/PcResultGenJune2024/partywisewinresult-369S02.htm')], 'Assam': [('Bharatiya Janata Party - BJP', 'https://results.eci.gov.in/PcResultGenJune2024/partywisewinresult-369S03.htm'), ('Indian National Congress - INC', 'https://results.eci.gov.in/PcResultGenJune2024/partywisewinresult-742S03.htm'), 

In [19]:
base_folder = 'state'
if not os.path.exists(base_folder):
    os.makedirs(base_folder)
for state, parties in state_party_links.items():
    state_folder = os.path.join(base_folder, state)
    if not os.path.exists(state_folder):
        os.makedirs(state_folder)
    

    for party, link in parties:
        party_folder = os.path.join(state_folder, party)
        if not os.path.exists(party_folder):
            os.makedirs(party_folder)

In [20]:
import requests
from bs4 import BeautifulSoup
import os
import csv

base_folder = 'state'

def fetch_and_parse(url):
    response = requests.get(url)
    if response.status_code == 200:
        return BeautifulSoup(response.content, 'html.parser')
    else:
        print(f"Failed to fetch {url}")
        return None

for state, parties in state_party_links.items():
    state_folder = os.path.join(base_folder, state)


    if not os.path.exists(state_folder):
        os.makedirs(state_folder)

    for party, link in parties:
        party_folder = os.path.join(state_folder, party)

        if not os.path.exists(party_folder):
            os.makedirs(party_folder)

        soup = fetch_and_parse(link)

        if soup:
            table = soup.find('table', class_='table table-striped table-bordered')

            if table:
                csv_filename = os.path.join(party_folder, f'{state}_{party}_election_results.csv')
                with open(csv_filename, mode='w', newline='', encoding='utf-8') as csvfile:
                    writer = csv.writer(csvfile)
                    
                    header_row = ['S.No', 'Parliament Constituency', 'Winning Candidate', 'Total Votes', 'Margin']
                    writer.writerow(header_row)

                    for row in table.find_all('tr')[1:]:
                        columns = row.find_all('td')
                        if len(columns) == 5:
                            s_no = columns[0].get_text().strip()
                            constituency = columns[1].get_text().strip()
                            candidate = columns[2].get_text().strip()
                            total_votes = columns[3].get_text().strip()
                            margin = columns[4].get_text().strip()
                            
                            writer.writerow([s_no, constituency, candidate, total_votes, margin])
                
                print(f"CSV file created for {state} - {party}: {csv_filename}")
            else:
                print(f"Table not found on {link}")
        else:
            print(f"Failed to fetch or parse {link}")


CSV file created for Andaman & Nicobar Islands - Bharatiya Janata Party - BJP: state\Andaman & Nicobar Islands\Bharatiya Janata Party - BJP\Andaman & Nicobar Islands_Bharatiya Janata Party - BJP_election_results.csv
CSV file created for Andhra Pradesh - Telugu Desam - TDP: state\Andhra Pradesh\Telugu Desam - TDP\Andhra Pradesh_Telugu Desam - TDP_election_results.csv
CSV file created for Andhra Pradesh - Yuvajana Sramika Rythu Congress Party - YSRCP: state\Andhra Pradesh\Yuvajana Sramika Rythu Congress Party - YSRCP\Andhra Pradesh_Yuvajana Sramika Rythu Congress Party - YSRCP_election_results.csv
CSV file created for Andhra Pradesh - Bharatiya Janata Party - BJP: state\Andhra Pradesh\Bharatiya Janata Party - BJP\Andhra Pradesh_Bharatiya Janata Party - BJP_election_results.csv
CSV file created for Andhra Pradesh - Janasena Party - JnP: state\Andhra Pradesh\Janasena Party - JnP\Andhra Pradesh_Janasena Party - JnP_election_results.csv
CSV file created for Arunachal Pradesh - Bharatiya Jana